# Лаб1. Расчет рейтингов фильмов – RDD

### Импорты и настройка окружения

In [1]:
import findspark
findspark.init('/opt/spark-3.4.3/')

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
# from pyspark.sql import functions as F
# from pyspark.sql.types import *
# from pyspark import Row

import json

conf = SparkConf()
spark = (
    SparkSession
    .builder
    .config(conf=conf)
    .appName('max_burdasov_lab1')
    .getOrCreate()
)

sc = spark.sparkContext

https://repos.spark-packages.org/ added as a remote repository with the name: repo-1
Ivy Default Cache set to: /data/home/maksim.burdasov/.ivy2/cache
The jars for the packages stored in: /data/home/maksim.burdasov/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0261ab85-6405-4a76-844d-36cc63b76dbf;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/spark-3.4.3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.3 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 469ms :: artifacts dl 11ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.apache.commons#commons-pool2;2.11.1 from central in [default]
	org.apache.hadoop#hadoop-client-api;3.3.4 from central in [default]
	org.apache.hadoop#

### Загрузка и агрегация данных

In [44]:
# Загрузка данных из HDFS, файл /labs/laba01/ml-100k/u.data
rdd = sc.textFile('/labs/laba01/ml-100k/u.data')

# Подсчет количества всех оценок
all_agg_ratings_lst = (
    rdd
    .map(lambda row: (row.split('\t')[2], 1))
    .reduceByKey(lambda res, next_val: res + next_val)
    .take(5)
)

# Подсчет количества оценок для фильма с заданным ID
target_film_id = '22'
trg_agg_ratings_lst = (
    rdd
    .filter(lambda row: row.split('\t')[1] == target_film_id)
    .map(lambda row: (row.split('\t')[2], 1))
    .reduceByKey(lambda res, next_val: res + next_val)
    .take(5)
)

### Форматирование и сохранение

In [49]:
# Сортировка по первому элементу (!порядок лексикографический)
all_agg_ratings_lst.sort()
trg_agg_ratings_lst.sort()

# Отформатированный результат
out_dict = {
   "hist_film": [cnt for _, cnt in trg_agg_ratings_lst],
   "hist_all": [cnt for _, cnt in all_agg_ratings_lst]
}

In [50]:
out_dict

{'hist_film': [5, 14, 46, 98, 134],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [55]:
# Сохранение результата
out_file_path = '/data/home/maksim.burdasov/lab01.json'

with open(out_file_path, 'w', encoding='utf8') as f:
    json.dump(out_dict, f, indent=4)

### Завершение контекста

In [56]:
spark.stop()